In [1]:
!pip install pymupdf
# !pip install python-docx
# !pip install python-pptx
# !pip install pypandoc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.5 MB/s eta 0:00:00


In [26]:
import fitz, os, re, pandas as pd
import zipfile
from lxml import etree

In [3]:
base_dir = 'drive/MyDrive/project-ugm/'
share_drive = 'drive/Shareddrives/'

In [55]:
def my_dict():
  my_df = {}

  for i in range(1, 15):
    my_df[f'box_{i}'] = '[{}]'

  return my_df

def extract_soal(strings):
  pattern = r'\d+\.\s.*'
  matches = re.findall(pattern, strings)

  return ''.join(matches).strip()

def extract_shapes_pdf(shape, page):
  rect = None

  if shape["type"] == "f":
    rect = fitz.Rect(shape["rect"])

  elif shape["type"] == "fs":
    rect = fitz.Rect(shape["rect"])

  return page.get_text("text", clip=rect)

def list_matakuliah(path):
  except_mk = ['Data.xlsx', 'Copy of Data.xlsx', 'Data Assignment.xlsx', 'Untitled document.gdoc']
  matakuliah = os.listdir(path)
  matakuliah = [matkul for matkul in matakuliah if matkul not in except_mk]

  return matakuliah

def list_tugas(how_much):
  return [f'tugas {i+1}' for i in range(how_much)]

def take_mk_tugas(lists, strings):
  result = next((item for item in lists if item in strings), None)
  return result

In [67]:
def extract_diagram_pdf(pdf_path):
  df = pd.DataFrame()
  my_arr = []

  doc = fitz.open(pdf_path)

  for page_number in range(len(doc)):
    page = doc[page_number]
    my_df = my_dict()
    my_df['answer_type'] = 'pdf/shapes'

    shapes = page.get_drawings()

    if len(shapes) > 5:
      asking_query = page.get_text("text")
      asking = extract_soal(asking_query)

      my_df['asking'] = asking if asking else "no_asking"
      id = 0

      for i in range(len(shapes)):
        text = extract_shapes_pdf(shapes[i], page)
        if text:
          id += 1
          my_df[f'box_{id}'] = text

        if i+1 == len(shapes):
          id = 0

      my_arr.append(my_df)

  df_table = pd.DataFrame(my_arr)
  df = pd.concat([df, df_table], axis=0, ignore_index=True)

  return df

def extract_diagram_docx(filename):
  with zipfile.ZipFile(docx_path, 'r') as docx_zip:
    with docx_zip.open('word/document.xml') as document_xml_file:
        document_xml = document_xml_file.read()

  try:
    root = etree.fromstring(document_xml)
    my_df = my_dict()
    df = pd.DataFrame()

    namespaces = {
        'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main',
        'a': 'http://schemas.openxmlformats.org/drawingml/2006/main',
    }

    drawings = root.findall('.//w:drawing', namespaces)

    for idx, drawing in enumerate(drawings):
      texts = drawing.findall('.//w:t', namespaces)
      combined_text = " ".join([text.text for text in texts])
      my_df[f'box_{idx+1}'] = combined_text

    df_table =  pd.DataFrame(my_df, columns=my_df.keys(), index=[0])
    df = pd.concat([df, df_table], axis=0, ignore_index=True)
    return df
  except:
    print(f'Error on the {filename} - Check it please')



def main(filename):
  if filename.endswith('.pdf'):
    df = extract_diagram_pdf(filename)
  elif filename.endswith('.docx'):
    df = extract_diagram_docx(filename)
    print(df)
  elif filename.endswith('.pptx'):
    df = pd.DataFrame
    # df = extract_images_from_pptx(filename)
  else:
    return None

  tasks = list_tugas(5)
  list_mk = list_matakuliah(os.path.join(share_drive, 'AES Dataset/Dataset'))
  nama_mapel = take_mk_tugas(list_mk, filename)
  nama_tugas = take_mk_tugas(tasks, filename.lower())

  df['nama_tugas'] = nama_tugas
  df['nama_mapel'] = nama_mapel
  df['filename'] = filename

  return df


In [68]:
pdf_path = os.path.join(share_drive, "AES Dataset/Dataset/Bahasa Indonesia/20212/FILE TUGAS/TUGAS 1/AKHSA AL ROFIU DEWANTO 042724106_2775428_assignsubmission_file_/New folder/TUGAS_BI_1_AKHSA AL ROFIU DEWANTO.pdf")
docx_path = os.path.join(base_dir, 'TUGAS_BI_1_AKHSA AL ROFIU DEWANTO.docx')
df = main(docx_path)
df

                                               box_1  \
0  Kongres Bahasa Indonesia ke-7 mengusulkan pemb...   

                                               box_2  \
0  Kongres Bahasa Indonesia ke-8 menyimpulkan   b...   

                                               box_3  \
0  Kongre s Bahasa Indonesia ke-9 kali ini   memb...   

                                               box_4  \
0  Kongres Bahasa Indonesia ke-10 kali ini diikut...   

                                               box_5 box_6 box_7 box_8 box_9  \
0  Kongres Bahasa Indonesia ke-11 mengusung tema ...  [{}]  [{}]  [{}]  [{}]   

  box_10 box_11 box_12 box_13 box_14  
0   [{}]   [{}]   [{}]   [{}]   [{}]  


,box_1,box_2,box_3,box_4,box_5,box_6,box_7,box_8,box_9,box_10,box_11,box_12,box_13,box_14,nama_tugas,nama_mapel,filename
0,Kongres Bahasa Indonesia ke-7 mengusulkan pemb...,Kongres Bahasa Indonesia ke-8 menyimpulkan b...,Kongre s Bahasa Indonesia ke-9 kali ini memb...,Kongres Bahasa Indonesia ke-10 kali ini diikut...,Kongres Bahasa Indonesia ke-11 mengusung tema ...,[{}],[{}],[{}],[{}],[{}],[{}],[{}],[{}],[{}],None,None,drive/MyDrive/project-ugm/TUGAS_BI_1_AKHSA AL ...


In [25]:
docx_path = os.path.join(base_dir, 'TUGAS_BI_1_AKHSA AL ROFIU DEWANTO.docx')

with zipfile.ZipFile(docx_path, 'r') as docx_zip:
    with docx_zip.open('word/document.xml') as document_xml_file:
        document_xml = document_xml_file.read()

root = etree.fromstring(document_xml)

namespaces = {
    'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main',
    'a': 'http://schemas.openxmlformats.org/drawingml/2006/main',
}

drawings = root.findall('.//w:drawing', namespaces)

diagram_texts = []
for drawing in drawings:
    texts = drawing.findall('.//w:t', namespaces)
    combined_text = " ".join([text.text for text in texts])
    diagram_texts.append(combined_text)

['Kongres Bahasa Indonesia ke-7 mengusulkan pembentukan Badan Pertimbangan Bahasa Indonesia. Salah   satu   topik   yang   dibahas   dalam kongres ini adalah memperkukuh kedudukan Bahasa Indonesia di era  globalisasi. Kongres   Bahasa   Indonesia   VII', 'Kongres Bahasa Indonesia ke-8 menyimpulkan   bahwa   berdasarkan   isi dari Kongres Sumpah Pemuda menyatakan bahwa para pemuda memiliki satu bahasa yakni Bahasa Indonesia dan bulan Oktober dijadikan menjadi Bulan Bahasa. Kongres   Bahasa   Indonesia   VIII', 'Kongre s Bahasa Indonesia ke-9 kali ini   membahas   lima   utama,   antara   lain : Bahasa Indonesia, bahasa Daerah, Penggunaan Bahasa Asing, Pengajaran   Bahasa   dan   Sastra,   serta Bahasa medua massa Kongres   Bahasa   Indonesia   IX', 'Kongres Bahasa Indonesia ke-10 kali ini diikuti oleh 1.168 peserta dari Indonesia   dan   dari   Luar   Negeri.   Salah satu yang topik dibahas dalam kongres ini adalah perlunya memantapkan kedudukan bahasa Indonesia melalui penerjemahan dan